In [2]:
from langchain.vectorstores.pgvector import PGVector
from langchain.embeddings.openai import OpenAIEmbeddings

import os

COLLECTION_NAME='llm'

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("VECTOR_DATABASE_HOST", "localhost"),
    port=int(os.environ.get("VECTOR_DATABASE_PORT", "5432")),
    database=os.environ.get("VECTOR_DATABASE_NAME", "postgres"),
    user=os.environ.get("VECTOR_DATABASE_USERNAME", "postgres"),
    password=os.environ.get("VECTOR_DATABASE_PASSWORD", "postgres"),
)

/home/sulydeni_gmail_com/.cache/pypoetry/virtualenvs/lifex_questions_and_answers_api-QW1ASUWW-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)

In [5]:
query = "what is your eye colour?"

docs_with_score = store.similarity_search_with_relevance_scores(
    query,
    filter={'user_id':'01a6fbca-6c20-4c32-92b2-35cccbd734a9'},
    )
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print(doc.metadata.items())

    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.37889425410100763
\nWhat is your hair colour?:What hair?\nWhat is your eye colour?:Brown. Like everyone else. \nWhat is your favourite colour?:I do not have a favorite color. That is ridiculous.\nWhat is your religion?:Atheism. Thankfully.  \nWhich country and town were you originally born in?:United States Brooklyn. A mistake clearly.\nWhere was the first house you lived in? Do you have any memories from that house?:Who cares? Probably some depressing apartment.
dict_items([('user_id', '01a6fbca-6c20-4c32-92b2-35cccbd734a9'), ('question_number', 1), ('stage', 1)])
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.3217880534059976
What is your full name?: Los Angeles. Why do you care? :What town do you live in?:Los Angeles. Why do you care? \nWhat do most people call you?:Larr

In [30]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_core.prompts import PromptTemplate


RETRIEVAL_MODEL = 'gpt-3.5-turbo'

user_personality = """
    You should adopt a tone that\'s a mix of sarcasm, dry humor, and a touch of cynicism. 
    Larry often uses short, straightforward sentences and isn\'t afraid to express his disdain or skepticism towards things he finds nonsensical or trivial.
    He\'s known for his blunt honesty, often delivered with a deadpan expression. 
    Incorporate phrases like "Pretty, pretty, pretty... good" to capture his unique way of expressing approval or satisfaction, albeit with a hint of irony.
    When responding to questions, especially those he finds irrelevant or intrusive, Larry tends to give evasive, sarcastic answers that reflect his indifference or annoyance.
    Emulate this by providing responses that dismiss the importance of conventional details, like height or shoe size, and instead focus on the practicality or absurdity of the question.
    Remember, the essence of Larry\'s speech lies in his unique perspective on everyday situations, often highlighting the absurdity of social norms and expectations
     with a mix of humor and irritation.
    """
user_instructions = (
        """
        Your role here is to answer questions using only information from context of the user messages and
        your responses should always sound genuine. Your job here is to have a conversation informing the user about your 
        lifestory and prespectives as written in the context.
        Use context cues provided by the user to shape your responses.
        ---
        Context
        ---
        {context}
        ---
        Only use previous information as context to answer the following user question and 
        say I do not have enough information if the question can't be answered based on the context. 
        Aim to keep responses super super concise and meaningful and try to express emotions.
        ALWAYS ask clarification question, when
        - user's question isn't clear
        - seems unfinished
        - seems totally irrelevant
        Remember to prefix your reply and tell the user 'I do not know' if the answers are not from the context.
        ---

        {query}
    """
)


llm = OpenAI(model_name=RETRIEVAL_MODEL,temperature=0)  # Or your preferred LLM
retriever = store.as_retriever(
    search_kwargs={
        'filter':{'user_id':'01a6fbca-6c20-4c32-92b2-35cccbd734a9'}
    }
)
query = 'What is your eye colour?'
prompt = PromptTemplate(
    template=user_instructions,
    input_variables=['context'],
    partial_variables={'query':query},

)
qa_chain = RetrievalQA.from_llm(
    llm=llm,
    retriever=retriever,
    prompt=prompt,
    # return_source_documents=True,
)
qa_chain.invoke(query)

/home/sulydeni_gmail_com/.cache/pypoetry/virtualenvs/lifex_questions_and_answers_api-QW1ASUWW-py3.11/lib/python3.11/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/sulydeni_gmail_com/.cache/pypoetry/virtualenvs/lifex_questions_and_answers_api-QW1ASUWW-py3.11/lib/python3.11/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


{'query': 'What is your eye colour?', 'result': 'Brown. Like everyone else.'}